## Quick (no inputs) analysis -- just need to vary arguments (below)

### Global Argument Variation:

In [ ]:
nb_sep_tit_i = '5' # PUT NUMBER FOR INITAL (J)NP SEPARATION

## COMMENT OR UNCOMMENT 2ND SET
nb_NvsC = ''
nb_NvsC_pn =''

#nb_NvsC = 'CONTROL'
#nb_NvsC_pn = '(C)'

equil_time = 300000 # TIME AT END OF EQUILIBRATION - FOR SEPARATING DATA

In [3]:
%pylab inline
from scipy.optimize import curve_fit


def extract_(filename, data_line = 1):
    '''
    INPUTS
    
    OUTPUTS
    '''
    # settings
    skiprows = 2
    
    data = loadtxt(filename, skiprows = skiprows)
    
    use_data = []
    for i in data:
        use_data.append(i[data_line])
        
    print('final timestep:',data[-1][0])
    print('number of data points:',len(use_data))

    output_list = [ use_data , data[-1][0] ]
    
    return output_list

def mean_(data):
    '''
    INPUTS
    
    OUTPUTS
    '''
    total = 0
    for i in data:
        total += i
        
    mean = total / len(data)
    return mean


def max(data):
    '''
    Gives maximum of a list or array of data
    '''
    maximum = data[0]
    for n in data:
        if maximum < n:
            maximum = n
    return maximum


def min(data):
    '''
    Gives minimum of a list or array of data
    '''
    minimum = data[0]
    for n in data:
        if minimum > n:
            minimum = n
    return minimum


def gr(fn, yl = '', tit = '',alt_fn ='', data_line = 1, av = True, err = False, start_time = 0, end_time = 'End', con = False):
    '''
    ALTERNATIVE Version 05/09/23
    Gives a graph from dat file
    Designed for LAMMPS simulation outputs
    Can output an average, or average w/ standard deviation
    
    Changes: DOES NOT ASK FOR INPUTS
             - Designed for repetitive data format, where notebook can be run to give resuts quicker
             REMOVED INPUTS:
             - filename (arg, fn)
             - timestep value (setting, ts)
             - y-axis label (arg, yl)
             - y-axis units (automatic)
             - title (arg, tit)
             - alternative filename (arg, alt_fn)
             
             ! NOW GRAPHING DEPENDS ON NOTEBOOK ITSELF!

    '''
    print()
    print('# # # # # #')
    print()
    print('Graph (alt) [Version 05/09/2023]')
    print()
    print('# # # # # #')
    print()
    
    # settings (~global) -- removes unnescesary arguments in ALT version
    units = 'real'
    loc = 'upper right'
    verticle = False
    elwt = 'Standard'
    ts = '1.0'
    
    # import data and output simple analysis (final timestep, number data points, mean) 
    
    ##filename = input('Filename (w/out .dat): ')
    filename = fn
    
    extraction = extract_(filename+'.dat', data_line)
    y = extraction[0]
    fts = extraction[1]
    
        # Graph annotations
    figure(figsize=[12,7.5])
    ##ts = input('Timestep Value: ')
    ft = float(ts)*fts
    xlabel('Time (fs) [Timestep = ' + ts + ']')

    ##yl = input('y-axis label: ')
    if yl == 'Temperature':
        ylu = 'K'
    elif yl == 'Volume':
        ylu = 'A$^{3}$'
    elif yl == 'Pressure':
        ylu = 'atm'
    elif yl == 'Enthalpy':
        ylu = 'kJ mol$^{-1}$'
    elif yl == 'Distance':
        ylu = 'A'
    elif yl == 'Force':
        ylu = 'kcal mol$^{-1}$ A$^{-1}$'
    else:
        ylu = ''
    ylabel(yl + ' (' + ylu + ')' )

    #t_i = input('Title: ')
    title(tit)
    
    pltlabel = 'Instantaneous ' + yl

    if start_time != 0 or end_time != 'End':
        start_data = int(round(start_time*len(y)/fts)) # if invoked by end_time argument, this should be 'inert'
        print('Starting data from data point', start_data)
        print('Ending with data point', end_time)
        if end_time == 'End': # if invoked by start_time argument, end_time should be 'inert'
            end_data = len(y)
        elif type(end_time) == int: # inputs for end_time that are not an integer or 'End' will break function
            end_data = int(round(end_time*len(y)/fts))
            ft = end_time
        y = y[start_data:end_data]
    
    if av == True:
        print('mean:',mean_(y))
    if err == True:
        print('Standard Deviation:', std(y))
    
    
    # output of graphs
    x = linspace(start_time,ft,len(y))
    plot(x, y, color='red',label=pltlabel, linewidth = 0.5)

    av_y_label = 'average ' + yl + ' = ' + str(round(mean_(y),1)) + ' ' + ylu
    
    if av == True and err == False:
        plot(linspace(start_time,ft,2),[mean_(y),mean_(y)],color='blue',label=av_y_label)

    elif err == True:
        elw = 10*len(y)/fts
        if elwt != 'Standard':
            elw = float(input('Error bar linewidth: '))
        mean_list = []
        for n in range(len(y)):
            mean_list.append(mean_(y))
        errorbar(x,mean_list,color='blue',label=av_y_label,yerr=std(y),elinewidth=elw)
        
    if verticle == True:
        xvert = float(input('Verticle x-axis position? '))
        plot([xvert,xvert],[min(y),max(y)],color='blue',linestyle='--',linewidth=0.5)
        
    # legend location and figure save
    legend(loc = loc)
    if con == False:
        savefig(filename + '.png')
    else:
        savefig(alt_fn + '.png')

        
def f1(x,m,c):
    '''returns straight line plot, where x is an array
    
    INPUTS
    x: array
    m: float, gradient
    c: float, y-intercept
    
    OUPUTS
    y: array
    '''
    y = x*m + c
    return y        


def fit_dat(fn_i, data_line = 1, f = f1):
    '''
    Fits straight line to a data - for AFTER equilibration ONLY
    [Version 07/09/23]
    
    INPUTS
    fn_i: string, the filename for loadtxt - NOTE: w/out the .dat
    data_line: int, index for desired column in .dat file - NOTE: index 0 useless
    f: function, required for curve_fit & plot - NOTE: [Version 07/09/23] must be f1 (y=m*x+c)
    
    OUPUTS
    print - fitted gradient 
    print - fitted y-intercept
    plot - straight line (y=m*x+c) of fitted parameters - NOTE: [Version 07/09/23] superimposed on gr, not saved in png 
    
    NOTE: equil_time must be defined GLOBALLY before use of fit_dat
    '''
    
    print()
    print('# # # # # #')
    print()
    print('Curve Fitting from fit_dat function [Version 07/09/23]')
    print()
    print('# # # # # #')
    print()
    
    extracted = extract_(fn_i + '.dat', data_line)
    
    forces_A_NEMD = extracted[0][int((equil_time/100)):]
    x_time = linspace(int((equil_time)), extracted[1], int(len(forces_A_NEMD)))
    
    fit = curve_fit(f,x_time,forces_A_NEMD,p0=[0,0])
    
    print('Fitted gradient =',fit[0][0])
    print('Fitted y-intercept =', fit[0][1])
    plot(x_time,f(x_time,fit[0][0],fit[0][1]),marker='',linestyle='--',color='black',label='y=mx+c fit')

Populating the interactive namespace from numpy and matplotlib


# Temperature

In [ ]:
fn_i = 'temperature'
yl_i = 'Temperature'
tit_i = 'Temperature over Dual JNP Simulation' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, av = False, start_time = 1400)

# Pressure

In [ ]:
fn_i = 'pressure'
yl_i = 'Pressure'
tit_i = 'Pressure over Dual JNP Simulation' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, av = False, start_time = 1400)

In [ ]:
fn_i = 'pressure'
yl_i = 'Pressure'
tit_i = 'Pressure (short) over Dual JNP Simulation ' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, av = False, start_time = 8000)

# Volume

In [ ]:
fn_i = 'volume'
yl_i = 'Volume'
tit_i = 'Volume over Dual JNP Simulation ' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, av = False)

In [ ]:
fn_i = 'volume'
yl_i = 'Volume'
tit_i = 'Volume (short) over Dual JNP Simulation ' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, av = False, start_time=8000)

# Separation

In [ ]:
fn_i = 'separation'
yl_i = 'Distance'
tit_i = 'Separation of NPs (starting ' + nb_sep_tit_i + ' A) - full ' + nb_NvsC
alt_fn_i = ''

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i)

In [ ]:
fn_i = 'separation'
yl_i = 'Distance'
tit_i = 'Separation of NPs (starting ' + nb_sep_tit_i + ' A) - NEMD ' + nb_NvsC
alt_fn_i = 'separation_n'

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i, con=True, start_time=equil_time)

# Forces

In [ ]:
fn_i = 'forces'
yl_i = 'Force'
tit_i = 'Force on NP "A" -- equilibration'
alt_fn_i = 'forces_A-equilibration'

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i, data_line = 1, err = True,end_time = equil_time ,con=True)

In [ ]:
fn_i = 'forces'
yl_i = 'Force'
tit_i = 'Force on NP "B" -- equilibration'
alt_fn_i = 'forces_B-equilibration'

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i, data_line = 2, err = True,end_time = equil_time,con=True)

In [ ]:
print(int(equil_time/100))

fn_i = 'forces'
yl_i = 'Force'
tit_i = 'Force on NP "A" -- "NEMD" ' + nb_NvsC
alt_fn_i = 'forces_A-NEMD' + nb_NvsC_pn

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i, data_line = 1, start_time = equil_time,con=True)
    
fit_dat(fn_i, data_line = 1)

In [ ]:
fn_i = 'forces'
yl_i = 'Force'
tit_i = 'Force on NP "B" -- "NEMD" ' + nb_NvsC
alt_fn_i = 'forces_B-NEMD' + nb_NvsC_pn

gr(fn=fn_i , yl=yl_i, tit=tit_i, alt_fn=alt_fn_i, data_line = 2, start_time = equil_time,con=True)

fit_dat(fn_i, data_line = 2)